# Predicting Car Crash Severity in the United States
### Analysis Completed by Vaibhav Sanjay, Kashish Patel

# Introduction

Car crashes are one of the largest non-illness causes of fatalities among people in the United States. Analysis of car accidents is useful for locating accident-prone regions, analyzing effects of weather, and predicting accidents. We analyze a countrywide dataset of car accidents to understand patterns in car crashes and predict the severity of them based on various parameters such as precipitation, visibility, temperature, and location.

We used the [US Accidents (2016-2021)](https://www.kaggle.com/datasets/sobhanmoosavi/us-accidents) dataset on Kaggle for our analysis [[1]](#1). The dataset includes 2.8 million points of accident data from February 2016 to December 2021 and covers 49 states.

<span style="color:#b00c00; font-weight:bold; font-size:larger">TO REMOVE</span>

_Dataset:
https://smoosavi.org/datasets/us_accidents_

Ideas:
- time graph
- ~~state frequency chloropleth~~

# Data Wrangling
This notebook depends on numpy, pandas, matplotlib, plotly, ....

## Reading in the Data
We start by doing the necessary imports and reading in the data. Note that there is a lot of data so this command and several subsequent commands may take some time to complete.

In [78]:
# imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import plotly.express as px

data_all = pd.read_csv('./US_Accidents_Dec21_updated.csv', nrows=100)
data_all.head()

,ID,Severity,Start_Time,End_Time,Start_Lat,Start_Lng,End_Lat,End_Lng,Distance(mi),Description,...,Roundabout,Station,Stop,Traffic_Calming,Traffic_Signal,Turning_Loop,Sunrise_Sunset,Civil_Twilight,Nautical_Twilight,Astronomical_Twilight
0,A-1,3,2016-02-08 00:37:08,2016-02-08 06:37:08,40.108910,-83.092860,40.112060,-83.031870,3.230,Between Sawmill Rd/Exit 20 and OH-315/Olentang...,...,False,False,False,False,False,False,Night,Night,Night,Night
1,A-2,2,2016-02-08 05:56:20,2016-02-08 11:56:20,39.865420,-84.062800,39.865010,-84.048730,0.747,At OH-4/OH-235/Exit 41 - Accident.,...,False,False,False,False,False,False,Night,Night,Night,Night
2,A-3,2,2016-02-08 06:15:39,2016-02-08 12:15:39,39.102660,-84.524680,39.102090,-84.523960,0.055,At I-71/US-50/Exit 1 - Accident.,...,False,False,False,False,False,False,Night,Night,Night,Day
3,A-4,2,2016-02-08 06:51:45,2016-02-08 12:51:45,41.062130,-81.537840,41.062170,-81.535470,0.123,At Dart Ave/Exit 21 - Accident.,...,False,False,False,False,False,False,Night,Night,Day,Day
4,A-5,3,2016-02-08 07:53:43,2016-02-08 13:53:43,39.172393,-84.492792,39.170476,-84.501798,0.500,At Mitchell Ave/Exit 6 - Accident.,...,False,False,False,False,False,False,Day,Day,Day,Day


The above dataframe has 47 columns. We will not be using all of these columns. Here are the descriptions for all the columns that we are interested in.
1. ID - .....

## Filtering Columns
We remove the rest of the columns. Here are explanations for why each was removed.
1. The "Description" column is too messy and does not provide much useful information.
2. The "Number", "Street", "Side", "County", "Zipcode", "Timezone", and "Airport_Code" columns all provide location information that is not as specific as latitude, longitude and city. The "Country" column is not useful since this data is for the USA.
3. The "Weather timestamp" column shows the timestamp that weather data is collected. This is not very significant.
4. The "Nautical_Twilight" and "Astronomical_Twilight" columns are different measure for when sunset occurs, which we decided to just use the sunset measure to keep everything consistent.

In [11]:

# get data
data = data_all.drop(
    ["Description", "Number", "Street", "Side", "County","Country", "Zipcode", 
    "Timezone", "Airport_Code", "Weather_Timestamp", "Nautical_Twilight", 
    "Astronomical_Twilight"],
    axis=1, 
    errors='ignore'
)

In [ ]:
# determine if both start AND end times are useful

# convert to datetimes
data['Start_Time'] = pd.to_datetime(data['Start_Time'])
data['End_Time'] = pd.to_datetime(data['End_Time'])

# check difference between start and end time
data['Time_Diff'] = data['End_Time'] - data['Start_Time']
data['Time_Diff'].describe()

# looks like we don't need endtime

## Encoding

In [ ]:
# one hot encoding cell
data['Roundabout'] = data['Roundabout'].apply(int)

In [7]:
data = data.drop(['Time_Diff', 'End_Time'], axis=1, errors='ignore')

# Data Exploration

## Graphing Accident Locations
We are given the state acronyms for every accident with the "State" column. Using this data, we create a heatmap for the frequency of car accidents across the United States.

In [100]:
# Change the value_counts into a DataFrame with each row being a state and its number of accidents
freqs = data['State'].value_counts().rename_axis('State').reset_index(name='Accidents')

# Create the figure
fig = px.choropleth(
        freqs,
        locations='State',
        color='Accidents',
        locationmode='USA-states',
        color_continuous_scale='reds',
        title='Frequency of Car Accidents in the US (2016-2021)',
        scope='usa'
)

# Show figure
fig.show()

# Citations
- <a id="1">[1]</a> 
Moosavi, Sobhan, Mohammad Hossein Samavatian, Srinivasan Parthasarathy, and Rajiv Ramnath. “A Countrywide Traffic Accident Dataset.”, 2019.
- <a id="2">[2]</a> 
Moosavi, Sobhan, Mohammad Hossein Samavatian, Srinivasan Parthasarathy, Radu Teodorescu, and Rajiv Ramnath. "Accident Risk Prediction based on Heterogeneous Sparse Data: New Dataset and Insights." In proceedings of the 27th ACM SIGSPATIAL International Conference on Advances in Geographic Information Systems, ACM, 2019.